# PMOIRED Tutorial 1
## How to fit diameters: uniform, limb-darkened, oblate, spotted, etc.

We will analyse the CHARA/MIRC data of CL Lac / IRC+50448, a Mira type AGB star. The data, which have been published in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract), and are available on [OIdB](https://oidb.jmmc.fr/search.html?conesearch=IRC%2B50448%2CJ2000%2C2%2Carcmin&perpage=50&instrument=MIRC&cs_radius_unit=arcmin&cs_equinox=J2000&order=t_min&caliblevel=3&category=SCIENCE&cs_radius=2&cs_position=IRC%2B50448). A subset of the data are provided with the tutorial.

In this tutorial, we will see how to:
- [load the data and display OIFITS data](#load)
- [fit a uniform disk model](#uniform_disk)
- [fit a limb-darkenend disk model](#limb-darkened_disk)
- [add a fully resolved component and oblatness to the star shape](#resolved)
- [add a spot on the star surface](#spot) 
- [use grid search to find the overall best position for the spot](#grid_search)
- [Bonus: use bootstrapping to evaluate uncertainties](#bootstrapping)
- [Bonus: try to come up with a realistic profile based on models](#realistic)

PMOIRED available at https://github.com/amerand/PMOIRED - tutorial by amerand@eso.org, Feb 2023

In [ ]:
%matplotlib widget
import os, sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
try:
    import pmoired
    print('global installation')
except:
    sys.path = ['../pmoired'] + sys.path
    import __init__ as pmoired
    print('local installation')

allfits = {}

# Load and preview files <a id='load'></a>

OIFITS data are in `./CL_Lac` (files names end in 'viscal.fits'). Use `oi = pmoired.OI(...)` to load the files and construct your object `oi`. the function `OI()` takes simply a list of file names. `oi.show()` will show the all the data. interesting options: 
- `logV=True` to show visibilities (amplitude or squared) in log scale.
- `showFlagged=True` to show flagged data (i.e. not taken into account).
- other options are described in the doc: `?pmoired.OI`

In [ ]:
directory = './CL_Lac/'
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('viscal.fits')]
display(files)
oi = pmoired.OI(files)
oi.show(logV=True) # possible with 'logV=True' to see low visibilities

# Uniform disk fit <a id='unfiform_disk'></a>

Using `oi.setupFit`, define the context of your fit using a dictionnary of options (check `?oi.setupFit`):
- declare that you fit the `V2` data. Use `'obs':['V2']`  
- you can use `'max relative error':{'V2':0.5}` to ignore the squared visibilities with relative uncertainites ($\sigma_{V^2}/V^2$) larger than 50%. 
- check `?oi.setupFit` to check all options

Then use `oi.doFit` to fit the data. `oi.doFit` takes as fist argument a dictionnary describing a model. By default, all parameters will be fitted. 

To see how write models as dictionnaries in `PMOIRED`, please refer to [the model definition notebook](https://github.com/amerand/PMOIRED/blob/master/examples/Model%20definitions%20and%20examples.ipynb). Let's start with a uniform disk of diameter 1.0 mas: `{'ud':1.0}`. Use different values of x as initial guess. It is better to start with a diameter too small than too large: try an initial guess >20mas. 

_As a general rule (for any parameters): do not start with exactly 0. The gradient descent algorithm has trouble choosing a step._ 

**LIMITATIONS**: the models will always disagree in the nulls ($B/\lambda\sim$100 and 200): this is because `PMOIRED` does not take into account  (yet) bandwidth smearing, which is when the visibility varies substantially within a spectral channel.

The result of the best fit is shown, and it can also be accessed in the dictionnary `oi.bestfit`. The basic informations are:
- `oi.bestfit['best']` contains the best model
- `oi.bestfit['uncer']` contains the uncertainties
- `oi.bestfit['chi2']` contains the final reduced $\chi^2$

In [ ]:
oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
oi.doFit({'ud':1}) # try several first values
allfits['UD'] = oi.bestfit.copy()
oi.show(logV=True)

# Limb darkened disk <a id='limb-dakened_disk'></a>
## using Claret (2000) 4-parameters

from [Claret (2000)](https://ui.adsabs.harvard.edu/abs/2000A%26A...363.1081C), table [J/A+A/363/1081/atlas](https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=J/A%2bA/363/1081/atlas&-out.max=50&-out.form=HTML%20Table&-out.add=_r&-out.add=_RAJ,_DEJ&-sort=_r&-oc.form=sexa) get the 4-coef CLD parameters: $I(\mu)/I(1) = 1-\sum_{k=1}^{4}a_k(1-\mu^{k/2})$. In this context, $\mu = \cos(\gamma) = \sqrt{1-r^2}$, $\gamma$ being the angle between the line of sight and the emergent intensity and $r$ the normalised radial distance from the centre of the star to its limb. 

Based on the stellar parameters in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract), you can use Teff=3500K and logg=1.0 in the H band for MIRCX which gives $a_1$=1.5899, $a_2$=-1.6835, $a_3$=1.0073, $a_4$=-0.2389

In `PMOIRED`, you describe a disk with arbitrary profile using `diam` and `profile`. The diameter is in milliarcseconds, and the profile is a string using special names `$R` and `$MU` and any additional parameters you need: for example, a linear limb-darkened disk, parametrised with `u`, will be entered in `PMOIRED` as `{'diam':2.0, 'profile':'1-$u*(1-$MU)', 'u':0.1}`.

You should first fix the LD parameters (the $a_k$) using the option `doNotFit=[...]` in `oi.doFit` to list the parameters you do not want to fit. In a second fit, you can try to fit them: the fit does not converge (see messages and the fact that uncertainties are not computed). You can inspect the fit with `oi.showfit()` which shows the evolution of the parameters during the fitting: use the mouse to zoom and inspect the convergence.

in `oi.show()`, we can display a synthetic image by giving a field-of-view parameter (in mas): `imFov=3`

In [ ]:
oi.setupFit({'obs':['V2'], 'max relative error':{'V2':0.5}})
# -- Teff=3500, logg=1.0, fixed parameters
oi.doFit({'diam':2.5, 
          'profile':'1 - $A1*(1-$MU**1/2) - $A2*(1-$MU**2/2) - $A3*(1-$MU**3/2) - $A4*(1-$MU**4/2)', 
          'A1':1.5899, 'A2':-1.6835, 'A3':1.0073, 'A4':-0.2389}, 
          doNotFit=['A1', 'A2', 'A3', 'A4'])
allfits['C2000 fixed'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, showUV=False)

# -- Teff=3500, logg=1.0, fit the 4 limb-darkening parameters (it does not converge)
oi.doFit({'diam':2.5, 
          'profile':'1 - $A1*(1-$MU**1/2) - $A2*(1-$MU**2/2) - $A3*(1-$MU**3/2) - $A4*(1-$MU**4/2)', 
          'A1':1.5899, 'A2':-1.6835, 'A3':1.0073, 'A4':-0.2389})
allfits['C2000 free'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, showUV=False)
oi.showFit()

## Adding prior to help fit the LD parameters

To help fit the LD profile, we can add the constrain that $|a_k|<2$ for instance. This is done using the `prior` keyword in `doFit`: we pass a list of priors as tuples: `prior=[('np.abs(A1)', '<', 2), ...]` (no $ where you refer to parameters!). 

Now that we successfully fot more that 1 parameters, `PMOIRED` shows the correlation matrix bewteew parameters. Idealy, one wants parameters to be uncorrelated. Getting parameters highly correlated (>~95% in absolute value) means that the fit is probably unreliable. Sometimes, one can re-parametrise the model to decrease the correlations (see tutorial #2).

Question: Can you tell if the fit is reliable? why?

In [ ]:
oi.setupFit({'obs':['V2'], 'max relative error':{'V2':0.5}})
m = {'diam':2.5, 
    'profile':'1 - $A1*(1-$MU**1/2) - $A2*(1-$MU**2/2) - $A3*(1-$MU**3/2) - $A4*(1-$MU**4/2)', 
    'A1':1.5899, 'A2':-1.6835, 'A3':1.0073, 'A4':-0.2389}
oi.doFit(m, prior=[('np.abs(A1)', '<', 2), ('np.abs(A2)', '<', 2), ('np.abs(A3)', '<', 2), ('np.abs(A4)', '<', 2)])
allfits['C2000 prior'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, showUV=False)

## Limb darkening: power law

To fit the limb darkening, we need a simpler law with less parameters: we can use a power law as described in [Hestroffer (1997)](https://ui.adsabs.harvard.edu/abs/1997A%26A...327..199H/abstract): $I(\mu)/I(1) = \mu^\alpha$. 

Compare the result in terms of reduced $\chi^2$ and correlation between parameters

In [ ]:
oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
prior=[('alpha', '>', 0)]
oi.doFit({'diam':2.5, 'profile':'$MU**$alpha', 'alpha':0.5}, prior=prior)
allfits['power law'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, showUV=False)

<a id='resolved'></a>
# Oblate star and resolved flux

We can refine the model:
- make the stellar shape oblate using `incl` and `projang`: `incl` is the "inclination", which means that the stellar shape will be an ellipse with large axis will have diameter `diam` and small axis cos(`incl`)$\times$`diam`. The large axis orientation is set by `projang`: 0 for North and 90 for East. All angles are in degrees. 
- add a fully resolved component (e.i. Visibility==0 at every baseline): just add a flux. Because we have 2 components, it is important to differentiate the componenents by using `name,parameter` in the model. A fully resolved component `res` is define just as a flux `{'res,f':...}`. Note that the star as an assumed total flux of 1, unless specified otherwise.

What justifies, in the data, to add oblatness to the star and a fully resolved component?  

In [ ]:
oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
prior=[('alpha', '>', 0)]
oi.doFit({'star,diam':2.5, 'star,profile':'$MU**$alpha', 'alpha':0.5, 'star,projang':45, 'star,incl':20, 'res,f':0.05},
           prior=prior)
allfits['oblate + resolved'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, showUV=False)

<a id='spot'></a>
# Adding a spot to the limb-darkened oblate model 

Add a spot to the previous best fit model (as a uniform disk for example). The spot must be able to be at different position on the star: use `x` and `y` (in mas). Also you should give it a flux `f` (total flux, not surface brightness!). 

Fit the `T3PHI` and `V2` data. if the spot gets too small, you can use a prior to force its size to be a reasonable fraction of the stellar size (between ~1/2 and ~1/4 of the size of the star, considering we have data in the third lobe).

You may find that the fit does not converge: it is because it is sensitive to the initial conditions, in particular the position of the spot. Try different initial positions.

In [ ]:
# -- we also fit the closure phase
oi.setupFit({'obs':['T3PHI', 'V2'],'max relative error':{'V2':0.5}, 
                'max error':{'T3PHI':30}, 
                })
# -- taking the best model previously fitted
m = {'alpha':       0.521, # +/- 0.017
    'res,f':       0.0202, # +/- 0.0020
    'star,diam':   2.6349, # +/- 0.0079
    'star,incl':   20.94, # +/- 0.62
    'star,projang':89.64, # +/- 1.62
    'star,profile':'$MU**$alpha',
    }
# -- adding a spot
m.update({'spot,diam':0.8, 'spot,x':0.1, 'spot,y':0.1, 'spot,f':0.1})

prior = [('alpha', '>', 0),
         ('spot,diam', '<', 'star,diam/4'), 
         ('spot,diam', '>', 'star,diam/8'),
         ('spot,x**2+spot,y**2', '<', 'max(star,diam/2 - spot,diam/2, 0)**2')]

oi.doFit(m, prior=prior)
# -- using imMax to be able to see the stellar surface
oi.show(imFov=3, logV=True, imMax='99', showUV=False)
oi.showFit()

## Randomized search to find the global best position for the spot <a id='grid_search'></a>


We saw the fit is sensitive to the initial position of the spot. Use `oi.gridFit()` to explore various initial positions. To define the exploration pattern, we use a dictionnary. see `?oi.gridFit` for more information, or the [notebook showing how to look for a companion around a star](https://github.com/amerand/PMOIRED/blob/master/examples/companion%20search%20AXCir.ipynb). In this case, a good choice is to use a 2D grid over the stellar surface. What should be the pitch of the grid? a fraction of the typical angular resolution (i.e. ~$\frac{\lambda}{3B}$). This will be checked *a posteriori* by the function using the number of unique minima compared to the number of initial guesses.

You can use as options in `oi.gridFit` (check `?oi.gridFit` for full description): 
- `constrain=` to give a list of constrain on the initial parameters, for example for the spot to fall strictly on the stellar surface
- `prior=` which will be used while optimising the model.

compare you result to the ones presented in [Fig 2](https://www.aanda.org/articles/aa/full_html/2020/08/aa37832-20/F2.html) of the publication. 

In [ ]:
# -- we also fit the closure phase
oi.setupFit({'obs':['T3PHI', 'V2'],
                'max relative error':{'V2':0.5}, # ignore large uncertainties
                'max error':{'T3PHI':30}, # ignore large uncertainties
                })
# -- taking the best model previously fitted
m = {'alpha':       0.521, # +/- 0.017
    'res,f':       0.0202, # +/- 0.0020
    'star,diam':   2.6349, # +/- 0.0079
    'star,incl':   20.94, # +/- 0.62
    'star,projang':89.64, # +/- 1.62
    'star,profile':'$MU**$alpha',
    }
# -- adding a spot
m.update({'spot,diam':0.8, 'spot,x':0.0, 'spot,y':0.0, 'spot,f':0.1})

# -- prior on the size of the spot
prior = [('alpha', '>', 0),
         ('spot,diam', '<', 'star,diam/4'), 
         ('spot,diam', '>', 'star,diam/8'),
         ('spot,x**2+spot,y**2', '<', 'max(star,diam/2 - spot,diam/2, 0)**2')]

# -- we define our exploration pattern (grid with step a fraction of angular resolution)
expl = {'grid':{'spot,x':(-m['star,diam']/2, m['star,diam']/2, 0.3), 
                'spot,y':(-m['star,diam']/2, m['star,diam']/2, 0.3)}}

# -- we constrain the exploration pattern so the spot is on the star
constrain = [('spot,x**2+spot,y**2', '<', '(star,diam/2)**2')]

# -- grid fit
oi.gridFit(expl, model=m, prior=prior, constrain=constrain)

allfits['oblate + resolved + spot'] = oi.bestfit.copy()

# -- show result of the grid:
oi.showGrid()

# -- show best fit model
oi.show(imFov=3, imMax='99', logV=1, showUV=False)

In [ ]:
from IPython.core.display import HTML
data = {'model': list(allfits.keys()),
        'chi2': [round(allfits[k]['chi2'], 2) for k in allfits],
        'parameters': [pmoired.oimodels.dpfit.dispBest(allfits[k], asStr=True, color=False).replace('$', '\$').replace('\n', 'bBRr').replace('\'', '"') 
                       for k in allfits]
       }
    
df = pd.DataFrame(data)
html = df.to_html(justify='left')
html = html.replace('bBRr', '<br>')

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: left;">
      <th></th>
      <th>model</th>
      <th>chi2</th>
      <th>parameters</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>UD</td>
      <td>47.34</td>
      <td>{"ud":2.5015, # +/- 0.0058<br>}<br></td>
    </tr>
    <tr style="text-align: left;">
      <th>1</th>
      <td>C2000 fixed</td>
      <td>26.35</td>
      <td>{"diam":   2.5320, # +/- 0.0047<br>"A1":     1.5899,<br>"A2":     -1.6835,<br>"A3":     1.0073,<br>"A4":     -0.2389,<br>"profile":"1 - \$A1*(1-\$MU**1/2) - \$A2*(1-\$MU**2/2) - \$A3*(1-\$MU**3/2) - \$A4*(1-\$MU**4/2)",<br>}<br></td>
    </tr>
    <tr>
      <th>2</th>
      <td>C2000 free</td>
      <td>13.31</td>
      <td>{"A1":     -16.27183574057675,<br>"A2":     21.007398368067566,<br>"A3":     12.86471061940458,<br>"A4":     -17.983257382104515,<br>"diam":   3.052814788478536,<br>"profile":"1 - \$A1*(1-\$MU**1/2) - \$A2*(1-\$MU**2/2) - \$A3*(1-\$MU**3/2) - \$A4*(1-\$MU**4/2)",<br>}<br></td>
    </tr>
    <tr>
      <th>3</th>
      <td>C2000 prior</td>
      <td>17.49</td>
      <td>{"A1":     -0.14, # +/- 4.79<br>"A2":     2.08, # +/- 0.18<br>"A3":     1.28, # +/- 0.42<br>"A4":     -2.25, # +/- 0.18<br>"diam":   2.79, # +/- 4.53<br>"profile":"1 - \$A1*(1-\$MU**1/2) - \$A2*(1-\$MU**2/2) - \$A3*(1-\$MU**3/2) - \$A4*(1-\$MU**4/2)",<br>}<br></td>
    </tr>
    <tr>
      <th>4</th>
      <td>power law</td>
      <td>20.18</td>
      <td>{"alpha":  0.477, # +/- 0.022<br>"diam":   2.5878, # +/- 0.0066<br>"profile":"\$MU**\$alpha",<br>}<br></td>
    </tr>
    <tr>
      <th>5</th>
      <td>oblate + resolved</td>
      <td>12.43</td>
      <td>{"alpha":       0.521, # +/- 0.017<br>"res,f":       0.0202, # +/- 0.0020<br>"star,diam":   2.6349, # +/- 0.0079<br>"star,incl":   20.94, # +/- 0.62<br>"star,projang":89.64, # +/- 1.62<br>"star,profile":"\$MU**\$alpha",<br>}<br></td>
    </tr>
    <tr>
      <th>6</th>
      <td>oblate + resolved + spot</td>
      <td>8.41</td>
      <td>{"alpha":       0.420, # +/- 0.021<br>"res,f":       0.0231, # +/- 0.0017<br>"spot,diam":   0.177, # +/- 0.083<br>"spot,f":      0.01584, # +/- 0.00072<br>"spot,x":      0.117, # +/- 0.011<br>"spot,y":      -0.423, # +/- 0.014<br>"star,diam":   2.6150, # +/- 0.0077<br>"star,incl":   18.13, # +/- 0.54<br>"star,projang":99.97, # +/- 1.49<br>"star,profile":"\$MU**\$alpha",<br>}<br></td>
    </tr>
  </tbody>
</table>



<a id='bootstrapping'></a>
# Bonus: Use bootstrapping to evaluate the uncertainties

Use `oi.bootstrapFit(Nfits)` to perform `Nfits` fit with resampled data. `Nfits` should be of the order of the (number of baselines + number of triangle)x(number of files). In our case, the is 540. Although the bootstrapping is parallelised, running 540 fits will take several minutes on a typical laptop. You can use a smaller number, e.g. 100, to get an idea of the result. `oi.showBootstrap()` let you see the result as a corner plot.

In [ ]:
oi.bootstrapFit(100)
oi.showBootstrap()

<a id='realistic'></a>
# Bonus: try to reproduce "realistic" limb darkening

The power law limb darkening has a sharp edge, whereas 3D models of AGB photosphere indicates that the egde is more "fuzzy" (see [Fig 4](https://www.aanda.org/articles/aa/full_html/2020/08/aa37832-20/F4.html) in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract)). we can use the flexibility of `'profile'` to create a similar profile. However, the $\chi^2$ is barelly better that the one with power law...

In [ ]:
# -- profile to look like Fig 4 in paper
p = {'star,diam':'(2+10/$k)*$Rstar', 'Rstar':1.5, 'res,f':0.02,
     'star,profile': '1-$u*(1-np.sqrt(1-(($R<$Rstar)*$R/$Rstar)**2)) + ($R>=$Rstar)*((1-$u)*np.exp(-$k*($R-$Rstar)/$Rstar)-1)',
     'u':0.2, 'k':10, 'star,projang':-90, 'star,incl':20}

oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
# -- cannot fit 'k', but 10 looks OK compared to fig 4
oi.doFit(p, doNotFit=['k'])

# -- show profile
plt.close(0); plt.figure(0)
p2 = pmoired.oimodels.computeLambdaParams(oi.bestfit['best'])
r = np.linspace(0, p2['star,diam']/2, 100)
plt.plot(r/(p2['Rstar']), eval(p2['star,profile'].replace('$R', 'r')))
plt.xlabel('r/R$_\star$')

# -- show data
oi.show(logV=1, imFov=1.1*p2['star,diam'], showUV=False)